In [230]:
#Downloading and formatting the dataset

from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

end_date = dt.datetime.now()
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

df = yf.download(tickers = symbols_list, start = start_date, end = end_date)
df = df.stack()
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()

[*********************100%***********************]  503 of 503 completed


In [231]:
#Computing the technical indicators

df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low'])) ** 2) / 2 - (2 * np.log(2) - 1) * ((np.log(df['adj close']) - np.log(df['open'])) ** 2)
df['rsi'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.rsi(close = x, length = 20))

df['bb_low'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,0])
df['bb_mid'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,1]) 
df['bb_high'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high = stock_data['high'],
                        low = stock_data['low'],
                        close = stock_data['close'],
                        length = 14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level = 1, group_keys = False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20)
    if macd is not None:
        return macd.iloc[:, 0].sub(macd.iloc[:, 0].mean()).div(macd.iloc[:, 0].std())
    else:
        return pd.Series([np.nan] * len(close), index=close.index)

df['macd'] = df.groupby(level = 1, group_keys = False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close'] * df['volume'])/1e6

In [232]:
#Aggregate to monthly level and filter top 150 most liquid stocks for each month
'''This is done to reduce training time for the ML model and experiment with strats'''
#This computes the average monthly dollar volume to give a sense of liquidity

last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume',
                                                          'open','high','low','close']]

data = pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack().to_frame('dollar_volume'),
          df.unstack()[last_cols].resample('M').last().stack('ticker')], axis = 1).dropna()

In [233]:
#Calculating the 5 year rolling average rolling average for each stock
#and use this value to filter out top 150 most liquid stocks for each month

data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = data.groupby('date')['dollar_volume'].rank(ascending = False)

#We are finding the 150 most liquid stocks then dropping the columns that we dont need
data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis = 1)

In [234]:
'''Removing the timezones from the dataframe data as they are not necessary'''
#Remove the time zone from the 'date' index of data
data.index = data.index.set_levels([data.index.levels[0].tz_localize(None), data.index.levels[1]])

#Convert the 'date' index to only the date part (remove time part)
data.index = data.index.set_levels([data.index.levels[0].date, data.index.levels[1]])

In [235]:
#Calculating monthly returns for different time horizons as features
'''To capture time series dynamics that reflect, for instance, momentum patterns, we
compute historical returns using the method .pct_change(lag), that is, returns over
various monthly periods as identified by lags'''

def calculate_returns(df):

    outlier_cutoff = 0.005 #only the 99.005 percentile is considered and values above this percentile are assigned the same value as the cutoff
    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:
        df[f'return_{lag}m'] = (df['adj close']
                            .pct_change(lag)
                            .pipe(lambda x: x.clip(lower = x.quantile(outlier_cutoff),
                                                    upper = x.quantile(1 - outlier_cutoff)))
                                .add(1)
                                .pow(1/lag)
                                .sub(1))
    return df
    
data = data.groupby(level = 1, group_keys = False).apply(calculate_returns).dropna()

In [236]:
'''Download FAMA-FRENCH factors and calculate rolling factor betas'''
#we are introducing the FAMA french data to estimate the exposure of assets to
#common risk factors with linear regression.
#five factors are market risk size, value, operating probability and investment
#these have been shown to empirically assess asset returns
#we can access historical returns using pandas-datareader and estimate historial 
#exposures using Rollingols model

factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
               'famafrench',
               start = '2010')[0].drop('RF', axis = 1)

factor_data.index = factor_data.index.to_timestamp() #but this returns beginning of month data, which we have to fix to end of month

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'
#now we just combine this with the 1 month return from previous code

# '''We are adding timezones to factordata date as data has timezones'''
# #Check if 'factor_data' is timezone-aware
# if factor_data.index.tz is None:
#     # If it's not timezone-aware, localize it to UTC (or any timezone you prefer)
#     factor_data.index = factor_data.index.tz_localize('UTC')
# else:
#     # If it's already timezone-aware, ensure it's in the same timezone (UTC)
#     factor_data.index = factor_data.index.tz_convert('UTC')
# '''end of bug fix'''

factor_data = factor_data.join(data['return_1m']).sort_index()

In [237]:
'''Filter out stocks with less than 10 months of data'''
#we do this because we are going to use rolling window for the regression of 24mnths
#stocks without enough data can break our functions

observations = factor_data.groupby(level = 1).size()
valid_stocks = observations[observations >= 10] #more than 10 months data

factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2018-11-30 AAPL    0.0169 -0.0077  0.0028 -0.0055  0.0039  -0.157549
           ABBV    0.0169 -0.0077  0.0028 -0.0055  0.0039   0.210919
           ABT     0.0169 -0.0077  0.0028 -0.0055  0.0039   0.074123
           ACN     0.0169 -0.0077  0.0028 -0.0055  0.0039   0.043776
           ADBE    0.0169 -0.0077  0.0028 -0.0055  0.0039   0.020874
...                   ...     ...     ...     ...     ...        ...
2024-09-30 VRTX    0.0174 -0.0103 -0.0260  0.0003 -0.0026  -0.062131
           VZ      0.0174 -0.0103 -0.0260  0.0003 -0.0026   0.074916
           WFC     0.0174 -0.0103 -0.0260  0.0003 -0.0026  -0.033864
           WMT     0.0174 -0.0103 -0.0260  0.0003 -0.0026   0.045578
           XOM     0.0174 -0.0103 -0.0260  0.0003 -0.0026  -0.006105

[10207 rows x 6 columns]

In [ ]:
'''Calculating the rolling factor betas'''

betas = (factor_data.groupby(level = 1,
                    group_keys = False)
        .apply(lambda x: RollingOLS(endog = x['return_1m'],
                                    exog = sm.add_constant(x.drop('return_1m', axis = 1)),
                                    window = min(24, x.shape[0]), #if less than 24 months data, still calculate ols
                                    min_nobs = len(x.columns)+1)
        .fit(params_only = True)
        .params
        .drop('const', axis = 1)))

Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2018-11-30 AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACN          NaN       NaN       NaN       NaN       NaN
           ADBE         NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2024-09-30 VRTX    0.889386  0.612210 -0.156810  1.237733 -0.294761
           VZ      0.552528 -0.785894  0.382449 -0.052894 -0.092445
           WFC     1.108949 -0.147554  1.287192 -0.756867 -0.751742
           WMT     0.694598  0.017539 -0.189687  1.177850  0.066447
           XOM     0.626831 -0.088954  0.428944  0.792986  0.802416

[10207 rows x 5 columns]

In [ ]:
'''Joining the betas to the features as well'''
#however we cannot blindly join betas to our dataset
#this is because we use the factor at the beginning of the month and return at the end of the month to compute betas.
#so at the end of the month, well be able to run the regression and have the betas but we will have them in the next month.
#so the betas of oct 31 will be known in nov 1
#so we shift the betas one month ahead before joining
#doing betas.shift() will simply shift betas forward by ticker, so XOM will get WMT's beta

data = data.join( betas.groupby('ticker').shift() )

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2018-11-30 AAPL     42.688763         -0.000970  36.859820  3.707130   
           ABBV     72.004761         -0.018581  60.417378  4.146937   
           ABT      66.759827         -0.003896  61.402894  4.124635   
           ACN     150.961502         -0.001608  54.580739  4.969324   
           ADBE    250.889999          0.000159  53.850578  5.393688   
...                       ...               ...        ...       ...   
2024-11-30 VRTX    502.450012          0.000517  60.672310  6.137945   
           VZ       40.570000          0.000078  40.156874  3.714518   
           WFC      69.820000          0.000117  67.562289  4.117228   
           WMT      83.849998          0.000066  63.699204  4.393277   
           XOM     121.150002          0.000122  55.534684  4.757746   

                     bb_mid   bb_high       atr      macd  return_1m  \
date       ticker                                                      
2018-11-30 AAPL    3.833351  3.959571 -0.635659 -1.334198  -0.157549   
           ABBV    4.218428  4.289920  0.378200  0.071550   0.210919   
           ABT     4.172122  4.219609 -0.205198  0.248628   0.074123   
           ACN     5.002466  5.035608 -0.652574 -0.188129   0.043776   
           ADBE    5.476070  5.558452 -0.101203 -0.300313   0.020874   
...                     ...       ...       ...       ...        ...   
2024-11-30 VRTX    6.178787  6.219629  2.409907  0.797798   0.055612   
           VZ      3.766371  3.818224 -0.159673 -1.299204  -0.037028   
           WFC     4.187614  4.258000  2.700757  2.738505   0.075478   
           WMT     4.419065  4.444852  1.684032  1.308370   0.023185   
           XOM     4.792950  4.828155  0.747137 -0.109417   0.037421   

                   return_2m  return_3m  return_6m  return_9m  return_12m  \
date       ticker                                                           
2018-11-30 AAPL    -0.109019  -0.076638  -0.006378   0.001491    0.004453   
           ABBV     0.003686  -0.002442  -0.004581  -0.019241    0.000878   
           ABT      0.006755   0.036152   0.032828   0.024552    0.024543   
           ACN     -0.012345  -0.006042   0.010714   0.004393    0.010356   
           ADBE    -0.035949  -0.016226   0.001074   0.020435    0.027361   
...                      ...        ...        ...        ...         ...   
2024-11-30 VRTX     0.039400   0.004390   0.016544   0.019916    0.029417   
           VZ      -0.042156  -0.004622   0.002926   0.006850    0.010160   
           WFC      0.111742   0.060919   0.027115   0.027169    0.039836   
           WMT      0.019014   0.027793   0.041829   0.041716    0.039981   
           XOM      0.016625   0.008991   0.006799   0.018362    0.015975   

                   Mkt-RF  SMB  HML  RMW  CMA  
date       ticker                              
2018-11-30 AAPL       NaN  NaN  NaN  NaN  NaN  
           ABBV       NaN  NaN  NaN  NaN  NaN  
           ABT        NaN  NaN  NaN  NaN  NaN  
           ACN        NaN  NaN  NaN  NaN  NaN  
           ADBE       NaN  NaN  NaN  NaN  NaN  
...                   ...  ...  ...  ...  ...  
2024-11-30 VRTX       NaN  NaN  NaN  NaN  NaN  
           VZ         NaN  NaN  NaN  NaN  NaN  
           WFC        NaN  NaN  NaN  NaN  NaN  
           WMT        NaN  NaN  NaN  NaN  NaN  
           XOM        NaN  NaN  NaN  NaN  NaN  

[10516 rows x 19 columns]